In [2]:
# import packages 
import rasterio
from rasterio.transform import from_origin
import os
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
from pathlib import Path
from rocksdbutils import *

def getFilelist(originpath, ftyp):
    files = os.listdir(originpath)
    out   = []
    for i in files:
        if i.split('.')[-1] in ftyp:
            if originpath.endswith('/'):
                out.append(originpath + i)
            else:
                out.append(originpath + '/' + i)
        # else:
        #     print("non-matching file - {} - found".format(i.split('.')[-1]))
    return out


In [3]:
# Set up export directory
export_dir = "/home/output/tiffs_back/ES"
origins = getFilelist("/home/ai4boundaries/sentinel2/masks/ES" , ['tif'])
origins.sort()

for modi in ['train', 'valid']:
    print(modi)
    db_path = '/home/output/rocks_db/ES.db/' + modi + '.db'
        
    # Initialize RocksDBReader
    reader = RocksDBReader(flname_db=db_path)

    # Iterate over data
    for idx in range(len(reader.keys)):
        # Get inputs and labels (assuming the database contains both)
        data = reader.get_inputs_labels(idx)  # This returns a list of arrays for inputs and labels

        # Process each column family
        for i, (cf_name, array) in enumerate(zip(reader.cf_names, data)):

            cf_name_str = cf_name.decode('utf-8')  # Convert bytes to string for readable file names
            print(f"cf name is: {cf_name_str} and shape is: {array.shape}")
        
            if cf_name_str == 'inputs':
                # Export to TIFF
                for t_idx in range(array.shape[0]):
                    out_file = f"{export_dir}/{cf_name_str}_image_{idx}_{modi}_{t_idx}.tif"
                    
                    with rasterio.open(
                        out_file,
                        "w",
                        driver="GTiff",
                        height=array.shape[2],
                        width=array.shape[3],
                        count=array.shape[1],
                        dtype=array.dtype,
                        crs=rasterio.open(origins[0]).crs,
                        transform=rasterio.open(origins[0]).transform,
                        ) as dst:
                                for r_idx in range(array.shape[1]):
                                    dst.write(array[t_idx, r_idx, :, :], r_idx + 1)

                    print(f"Exported {cf_name_str} index {idx} to {out_file}")
    
            else:
                 # Export to TIFF
                out_file = f"{export_dir}/{cf_name_str}_image_{idx}_{modi}.tif"
                
                with rasterio.open(
                    out_file,
                    "w",
                    driver="GTiff",
                    height=array.shape[1],
                    width=array.shape[2],
                    count=array.shape[0],
                    dtype=array.dtype,
                    crs=rasterio.open(origins[0]).crs,
                    transform=rasterio.open(origins[0]).transform,
                    ) as dst:
                            for r_idx in range(array.shape[0]):
                                dst.write(array[r_idx, :, :], r_idx + 1)

                print(f"Exported {cf_name_str} index {idx} to {out_file}")
                print(array.shape)
        if idx == 50:
            break


train
cf name is: inputs and shape is: (4, 6, 128, 128)
Exported inputs index 0 to /home/output/tiffs_back/ES/inputs_image_0_train_0.tif
Exported inputs index 0 to /home/output/tiffs_back/ES/inputs_image_0_train_1.tif
Exported inputs index 0 to /home/output/tiffs_back/ES/inputs_image_0_train_2.tif
Exported inputs index 0 to /home/output/tiffs_back/ES/inputs_image_0_train_3.tif
cf name is: labels and shape is: (4, 128, 128)
Exported labels index 0 to /home/output/tiffs_back/ES/labels_image_0_train.tif
(4, 128, 128)
cf name is: inputs and shape is: (4, 6, 128, 128)
Exported inputs index 1 to /home/output/tiffs_back/ES/inputs_image_1_train_0.tif
Exported inputs index 1 to /home/output/tiffs_back/ES/inputs_image_1_train_1.tif
Exported inputs index 1 to /home/output/tiffs_back/ES/inputs_image_1_train_2.tif
Exported inputs index 1 to /home/output/tiffs_back/ES/inputs_image_1_train_3.tif
cf name is: labels and shape is: (4, 128, 128)
Exported labels index 1 to /home/output/tiffs_back/ES/label